# Tutorial 2 - Hierarchical Agents

## Functionalities:
- An agent (referred as meta agent) can also assign other agents (referred as inner agents) as the function call, thereby creating a hierarchical structure
- Whenever the inner agent is called, we will run the full inner agent's process until termination
- Maximum number of steps for each agent (meta agent or inner agent) can be defined in `max_subtasks` (default: 5) variable when initialising agent

## Additional Details:
- The inner agent will take in meta agent's instruction as input
- In addition, the inner agent will also have access to meta agent's task and subtasks completed for global context
- Inner agent's output will directly go into meta agent's subtasks completed dictionary, enabling it to directly update global context

## Design Philosophy:
- Bottom layers can get context of the top layers for more context-specific generation
- Top layers choose bottom layers in a hierarchical fashion so as to limit the number of choices at each level for better performance
- `subtasks_completed` serves as the global context for the meta agents and the inner agents, and can be summarised as needed, e.g. when performing a new task to filter out irrelevant information 

# Install TaskGen

In [1]:
# !pip install taskgen-ai

In [2]:
# Set up API key and do the necessary imports
import os
from taskgen import *

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Define Agents and Meta Agents
- You can also create a Meta Agent that uses other Agents (referred to as Inner Agents) as functions
- Create your Meta agent using `Agent()` (Note: No different from usual process of creating Agents - your Meta Agent is also an Agent)
- Set up an Inner Agent list and assign it to your Meta agent using `assign_agents(agent_list)`

## Example Meta Agent Setup
```python
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Does final quality check on menu items'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them').assign_functions([fp])
    ]

my_agent.assign_agents(agent_list)
```

## Run the Meta Agent
- Let us run the agent and see the interactions between the Meta Agent and Inner Agents to solve the task!
```python
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.')
```

In [3]:
def random_number_from_string(input_string):
    ''' Returns a random number between 1 and 10 when given an input string '''
    import hashlib
    import random
    # Hash the input string
    hashed_string = hashlib.sha256(input_string.encode()).hexdigest()
    
    # Convert the hashed string to an integer
    hashed_integer = int(hashed_string, 16)
    
    # Seed the random number generator with the hashed integer
    random.seed(hashed_integer)
    
    # Generate a random number between 1 and 10
    return random.randint(1, 10)

In [4]:
# assign a dish pricing function for economist
# Note: Try to make the output informative that is understandable by the Agent - use free text or json to explain the output so that someone can understand without the input
def dish_price(list_of_dish_names: list) -> dict:
    ''' Takes in list_of_dish_names and outputs price '''
    if not isinstance(list_of_dish_names, list):
        list_of_food_items = [list_of_dish_names]
    output = {}
    for each in list_of_dish_names:
        output[each] = '$'+str(random_number_from_string(each))
    return output
    
fp = Function(fn_description = "Takes in <list_of_dish_names: list> and outputs price", 
              output_format = {"Price": "Dish name as key and price as value"}, 
              examples = {"list_of_dish_names": ["Pasta", "Pizza"], "Price": [{'Pasta': '$5'}, {'Pizza':'$2'}]},
              external_fn = dish_price)

In [5]:
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

In [6]:
# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Does final quality check on menu items'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', max_subtasks = 1),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them').assign_functions([fp])
    ]

In [7]:
my_agent.assign_agents(agent_list)

# Approach 1: Run a generic task with the Meta-Agent
- Let us run a general task to create 5 menu items for an Italian restaurant
- You can also specify how Agents interact with one another in the task, otherwise the Meta-Agent will infer what should be done

In [8]:
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.')

Subtask identified: Use Creative Writer to generate dish names and descriptions for Italian food choices
Calling function Creative Writer with parameters {'instruction': 'Use Creative Writer to generate dish names and descriptions for Italian food choices'}

### Start of Inner Agent: Creative Writer ###
Subtask identified: Use Creative Writer to generate dish names and descriptions for Italian food choices
Getting LLM to perform the following task: Use Creative Writer to generate dish names and descriptions for Italian food choices
> {'Menu': [{'name': 'Ravioli di Zucca', 'description': 'Homemade ravioli filled with roasted pumpkin, ricotta cheese, and sage butter sauce.'}, {'name': 'Pollo alla Cacciatora', 'description': 'Tender chicken cooked in a rich tomato sauce with onions, peppers, and mushrooms.'}, {'name': 'Tiramisu Classico', 'description': 'Traditional Italian dessert made with layers of coffee-soaked ladyfingers and mascarpone cream, dusted with cocoa powder.'}, {'name': "P

In [9]:
output = my_agent.reply_user()

I have created a menu with 5 Italian food choices. Each menu item includes a name, description, ingredients, and pricing. Here are the menu items:
1. Name: Ravioli di Zucca
   Description: Homemade ravioli filled with roasted pumpkin, ricotta cheese, and sage butter sauce.
   Ingredients: Roasted pumpkin, ricotta cheese, sage, butter, flour, eggs, salt, pepper
   Pricing: Not provided
2. Name: Pollo alla Cacciatora
   Description: Tender chicken cooked in a rich tomato sauce with onions, peppers, and mushrooms.
   Ingredients: Chicken, tomato sauce, onions, peppers, mushrooms, olive oil, garlic, salt, pepper, herbs
   Pricing: Not provided
3. Name: Tiramisu Classico
   Description: Traditional Italian dessert made with layers of coffee-soaked ladyfingers and mascarpone cream, dusted with cocoa powder.
   Ingredients: Ladyfingers, mascarpone cheese, eggs, sugar, coffee, cocoa powder
   Pricing: Not provided
4. Name: Penne all'Arrabbiata
   Description: Penne pasta tossed in a spicy toma

In [10]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed:
Subtask: Use Creative Writer to generate dish names and descriptions for Italian food choices
{'Output': {'Menu': [{'name': 'Ravioli di Zucca', 'description': 'Homemade ravioli filled with roasted pumpkin, ricotta cheese, and sage butter sauce.'}, {'name': 'Pollo alla Cacciatora', 'description': 'Tender chicken cooked in a rich tomato sauce with onions, peppers, and mushrooms.'}, {'name': 'Tiramisu Classico', 'description': 'Traditional Italian dessert made with layers of coffee-soaked ladyfingers and mascarpone cream, dusted with cocoa powder.'}, {'name': "Penne all'Arrabbiata", 'description': 'Pe

# Approach 2: Step through the process yourself as the Meta Agent
- You can use agents manually using `use_agent(agent_name: str, agent_task: str)` The outputs of the agents will be stored into `subtasks_completed` of the meta agent automatically
- You can also interact with the meta agent using `reply_user(task: str, stateful: bool = False)`. This generates a reply for the agent to the `task` using `subtasks_completed` as context. If `stateful` is true, this will also store the output into `subtasks_completed`
- If `subtasks_completed` is too lengthy, you can run `summarise_subtasks_completed(task: str)` This summarises `subtasks_completed` according to what is relevant to the `task`
- If you want to redo a subtask, you can remove the most recent subtask from `subtasks_completed` using `remove_last_subtask()`

In [11]:
# reset the state of the agent
my_agent.reset()
# currently should have no task and nothing in subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [12]:
# Assign a task for the meta agent - this helps to contextualise what you give each of the inner agents later on
my_agent.assign_task('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.')

In [13]:
# This will run the Agent Creative Writer (with shared subtasks_completed as meta agent)
# Output of task will go directly into subtasks_completed - we view the inner agent as simply an extension of the meta agent
my_agent.use_agent('Creative Writer', 'Generate only 5 Italian dish names and descriptions')

Calling function Creative Writer with parameters {'instruction': 'Generate only 5 Italian dish names and descriptions'}

### Start of Inner Agent: Creative Writer ###
Subtask identified: Generate Italian dish names and descriptions
Getting LLM to perform the following task: Generate Italian dish names and descriptions
> {'Menu Items': [{'Name': 'Ravioli di Zucca', 'Description': 'Homemade ravioli filled with roasted pumpkin, ricotta cheese, and sage butter sauce.'}, {'Name': 'Pollo alla Cacciatora', 'Description': 'Tender chicken cooked in a rich tomato sauce with onions, mushrooms, and bell peppers.'}, {'Name': 'Gnocchi al Gorgonzola', 'Description': 'Potato gnocchi served in a creamy gorgonzola cheese sauce with walnuts and fresh parsley.'}, {'Name': 'Osso Buco alla Milanese', 'Description': 'Braised veal shanks cooked in white wine and broth, served with gremolata and risotto alla Milanese.'}, {'Name': 'Tiramisu', 'Description': 'Classic Italian dessert made with layers of coffee-so

In [14]:
# subtasks_completed should be updated with the Creative Agent's outputs
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed:
Subtask: Generate Italian dish names and descriptions
{'Output': {'Menu Items': [{'Name': 'Ravioli di Zucca', 'Description': 'Homemade ravioli filled with roasted pumpkin, ricotta cheese, and sage butter sauce.'}, {'Name': 'Pollo alla Cacciatora', 'Description': 'Tender chicken cooked in a rich tomato sauce with onions, mushrooms, and bell peppers.'}, {'Name': 'Gnocchi al Gorgonzola', 'Description': 'Potato gnocchi served in a creamy gorgonzola cheese sauce with walnuts and fresh parsley.'}, {'Name': 'Osso Buco alla Milanese', 'Description': 'Braised veal shanks cooked in white wine and broth, serv

In [15]:
my_agent.use_agent('Chef', 'Generate ingredients for the 5 Italian dishes')

Calling function Chef with parameters {'instruction': 'Generate ingredients for the 5 Italian dishes'}

### Start of Inner Agent: Chef ###
Subtask identified: Generate ingredients for the 5 Italian dishes
Getting LLM to perform the following task: Generate ingredients for the 5 Italian dishes
> As Chef, I have generated the ingredients for the 5 Italian dishes: 
1. Ravioli di Zucca: Roasted pumpkin, ricotta cheese, sage, butter, flour, eggs
2. Pollo alla Cacciatora: Chicken, tomato sauce, onions, mushrooms, bell peppers, olive oil
3. Gnocchi al Gorgonzola: Potato, gorgonzola cheese, walnuts, parsley, cream, butter
4. Osso Buco alla Milanese: Veal shanks, white wine, broth, gremolata, risotto, saffron
5. Tiramisu: Ladyfingers, coffee, mascarpone cheese, cocoa powder, eggs, sugar

Task completed successfully!

### End of Inner Agent: Chef ###

> {'Status': 'Completed'}



In [16]:
my_agent.use_agent('Economist', 'Generate prices for the 5 Italian dishes')

Calling function Economist with parameters {'instruction': 'Generate prices for the 5 Italian dishes'}

### Start of Inner Agent: Economist ###
Subtask identified: Determine the pricing for the 5 Italian dishes
Calling function dish_price with parameters {'list_of_dish_names': ['Ravioli di Zucca', 'Pollo alla Cacciatora', 'Gnocchi al Gorgonzola', 'Osso Buco alla Milanese', 'Tiramisu']}
> {'Price': {'Ravioli di Zucca': '$5', 'Pollo alla Cacciatora': '$6', 'Gnocchi al Gorgonzola': '$9', 'Osso Buco alla Milanese': '$2', 'Tiramisu': '$10'}}

Subtask identified: Generate prices for the 5 Italian dishes
Calling function dish_price with parameters {'list_of_dish_names': ['Ravioli di Zucca', 'Pollo alla Cacciatora', 'Gnocchi al Gorgonzola', 'Osso Buco alla Milanese', 'Tiramisu']}
> {'Price': {'Ravioli di Zucca': '$5', 'Pollo alla Cacciatora': '$6', 'Gnocchi al Gorgonzola': '$9', 'Osso Buco alla Milanese': '$2', 'Tiramisu': '$10'}}

Task completed successfully!

### End of Inner Agent: Economis

In [17]:
my_agent.use_agent('Boss', 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay')

Calling function Boss with parameters {'instruction': 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay'}

### Start of Inner Agent: Boss ###
Subtask identified: Select 3 Italian dishes that will please Gordon Ramsay
Getting LLM to perform the following task: Select 3 Italian dishes that will please Gordon Ramsay
> As the final quality check agent, I have selected 3 Italian dishes that will please Gordon Ramsay: 
1. Ravioli di Zucca: Homemade ravioli filled with roasted pumpkin, ricotta cheese, and sage butter sauce.
2. Gnocchi al Gorgonzola: Potato gnocchi served in a creamy gorgonzola cheese sauce with walnuts and fresh parsley.
3. Tiramisu: Classic Italian dessert made with layers of coffee-soaked ladyfingers and mascarpone cheese, dusted with cocoa powder. These dishes are sure to impress Gordon Ramsay with their flavors and presentation.

Task completed successfully!

### End of Inner Agent: Boss ###

> {'Status': 'Completed'}



In [18]:
# Get a targeted response from your agent, and add this to subtask_completed
output = my_agent.reply_user('Choose the 3 menu items for Gordon Ramsay line by line in this format - Dish Name, Dish Description, Dish Ingredients, Dish Price')

1. Ravioli di Zucca, Homemade ravioli filled with roasted pumpkin, ricotta cheese, and sage butter sauce, Roasted pumpkin, ricotta cheese, sage, butter, flour, eggs, $5
2. Gnocchi al Gorgonzola, Potato gnocchi served in a creamy gorgonzola cheese sauce with walnuts and fresh parsley, Potato, gorgonzola cheese, walnuts, parsley, cream, butter, $9
3. Tiramisu, Classic Italian dessert made with layers of coffee-soaked ladyfingers and mascarpone cheese, dusted with cocoa powder, Ladyfingers, coffee, mascarpone cheese, cocoa powder, eggs, sugar, $10


In [19]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed:
Subtask: Generate Italian dish names and descriptions
{'Output': {'Menu Items': [{'Name': 'Ravioli di Zucca', 'Description': 'Homemade ravioli filled with roasted pumpkin, ricotta cheese, and sage butter sauce.'}, {'Name': 'Pollo alla Cacciatora', 'Description': 'Tender chicken cooked in a rich tomato sauce with onions, mushrooms, and bell peppers.'}, {'Name': 'Gnocchi al Gorgonzola', 'Description': 'Potato gnocchi served in a creamy gorgonzola cheese sauce with walnuts and fresh parsley.'}, {'Name': 'Osso Buco alla Milanese', 'Description': 'Braised veal shanks cooked in white wine and broth, serv

In [20]:
# you can summarise the subtasks_history for the next task
my_agent.summarise_subtasks_completed('Generate the 3 Italian dishes for Gordon Ramsay in menu format')

{
1. Name: Ravioli di Zucca
   Description: Homemade ravioli filled with roasted pumpkin, ricotta cheese, and sage butter sauce.
   Ingredients: Roasted pumpkin, ricotta cheese, sage, butter, flour, eggs
   Price: $5

2. Name: Gnocchi al Gorgonzola
   Description: Potato gnocchi served in a creamy gorgonzola cheese sauce with walnuts and fresh parsley.
   Ingredients: Potato, gorgonzola cheese, walnuts, parsley, cream, butter
   Price: $9

3. Name: Tiramisu
   Description: Classic Italian dessert made with layers of coffee-soaked ladyfingers and mascarpone cheese, dusted with cocoa powder.
   Ingredients: Ladyfingers, coffee, mascarpone cheese, cocoa powder, eggs, sugar
   Price: $10
}


In [21]:
# subtask has been summarised, helping to reduce the context significantly for later task generation
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed:
Subtask: Summary of Generate the 3 Italian dishes for Gordon Ramsay in menu format
{
1. Name: Ravioli di Zucca
   Description: Homemade ravioli filled with roasted pumpkin, ricotta cheese, and sage butter sauce.
   Ingredients: Roasted pumpkin, ricotta cheese, sage, butter, flour, eggs
   Price: $5

2. Name: Gnocchi al Gorgonzola
   Description: Potato gnocchi served in a creamy gorgonzola cheese sauce with walnuts and fresh parsley.
   Ingredients: Potato, gorgonzola cheese, walnuts, parsley, cream, butter
   Price: $9

3. Name: Tiramisu
   Description: Classic Italian dessert made with layers of 

In [22]:
# we also allow you to manually remove the last item added to subtasks_completed, so you can revert your agent state if needed for backtracking
my_agent.remove_last_subtask()

Removed last subtask from subtasks_completed: ('Summary of Generate the 3 Italian dishes for Gordon Ramsay in menu format', '{\n1. Name: Ravioli di Zucca\n   Description: Homemade ravioli filled with roasted pumpkin, ricotta cheese, and sage butter sauce.\n   Ingredients: Roasted pumpkin, ricotta cheese, sage, butter, flour, eggs\n   Price: $5\n\n2. Name: Gnocchi al Gorgonzola\n   Description: Potato gnocchi served in a creamy gorgonzola cheese sauce with walnuts and fresh parsley.\n   Ingredients: Potato, gorgonzola cheese, walnuts, parsley, cream, butter\n   Price: $9\n\n3. Name: Tiramisu\n   Description: Classic Italian dessert made with layers of coffee-soaked ladyfingers and mascarpone cheese, dusted with cocoa powder.\n   Ingredients: Ladyfingers, coffee, mascarpone cheese, cocoa powder, eggs, sugar\n   Price: $10\n}')


In [23]:
# visualise your agent's status - should have empty subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed: None
Is Task Completed: False
